---
## Simple generator example reading midi files
### Monday: 26-02-2024

* Read the MIDI file normally
    * We are given a list with all midi elements

* Read the MIDI file using the generator 
    * We are "`yield`ed" each element individually
    * We can get elements with `next()` or with a `loop`

---

In [1]:
#To help us import from outside folder
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from generatorReadMidi import GeneratorReadMidi

midi_file_path = "../data/midi/test.mid"

generator = GeneratorReadMidi()

# Read the MIDI file normally
data = generator.readMidi(midi_file_path)
print(data)

# Read the MIDI file using the generator 
data = generator.readMidiGenerator(midi_file_path)
for d in data:
    print(d)




[MetaMessage('smpte_offset', frame_rate=30, hours=0, minutes=0, seconds=0, frames=0, sub_frames=0, time=0), MetaMessage('time_signature', numerator=3, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0), MetaMessage('key_signature', key='Bb', time=0), MetaMessage('set_tempo', tempo=499999, time=0), MetaMessage('set_tempo', tempo=666669, time=0), MetaMessage('set_tempo', tempo=666669, time=0), MetaMessage('device_name', name='SmartMusic SoftSynth 1', time=0), MetaMessage('track_name', name='Piano', time=0), Message('program_change', channel=0, program=1, time=0), Message('control_change', channel=0, control=7, value=101, time=0), Message('control_change', channel=0, control=10, value=64, time=0), Message('control_change', channel=0, control=7, value=102, time=0), Message('control_change', channel=0, control=7, value=110, time=0), Message('control_change', channel=0, control=100, value=0, time=0.0019531318359375), Message('control_change', channel=0, control=101, v